# SQL Translation Model Benchmark - Interactive

Simple notebook interface to run benchmarks using the `run_local_benchmark.py` script.

## What This Does

- 🤖 Compares SQL translation models (Llama 4 Maverick vs Gemini 2.5 Flash)
- ⚖️ Uses LLM-as-judge to evaluate Databricks SQL compliance
- 📊 Tracks results in Databricks MLflow
- 📈 Shows comparison charts

## 1. Configuration

Edit these settings to customize your benchmark:

In [ ]:
# ═══════════════════════════════════════════════════════════════
# CONFIGURATION - Customize these settings
# ═══════════════════════════════════════════════════════════════

# Models to benchmark (for SQL translation)
TRANSLATION_MODELS = [
    "databricks-llama-4-maverick",
    "databricks-gemini-2-5-flash",
]

# Judge model (evaluates SQL compliance)
JUDGE_MODEL = "databricks-llama-4-maverick"

# Artifact type to benchmark
ARTIFACT_TYPE = "tables"  # Options: tables, views, procedures

# Batch size for processing
BATCH_SIZE = 5

# Optional: Custom dataset path (None = use built-in examples)
DATASET_PATH = None  # e.g., "../examples/tables.json"

print("📋 Configuration:")
print(f"  Translation models: {TRANSLATION_MODELS}")
print(f"  Judge model: {JUDGE_MODEL}")
print(f"  Artifact type: {ARTIFACT_TYPE}")
print(f"  Batch size: {BATCH_SIZE}")

## 2. Run Benchmark

Execute the benchmark with your configuration:

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
while not (project_root / "run_local_benchmark.py").exists() and project_root.parent != project_root:
    project_root = project_root.parent

if not (project_root / "run_local_benchmark.py").exists():
    print("❌ Could not find run_local_benchmark.py")
    print(f"   Searched from: {Path.cwd()}")
    print(f"   Ended at: {project_root}")
else:
    print(f"✅ Found project root: {project_root}")

In [ ]:
# Build command arguments
import subprocess

cmd = [
    "python3",
    str(project_root / "run_local_benchmark.py"),
    "--artifact-type", ARTIFACT_TYPE,
    "--batch-size", str(BATCH_SIZE),
    "--judge-endpoint", JUDGE_MODEL,
    "--models"
] + TRANSLATION_MODELS

if DATASET_PATH:
    cmd.extend(["--dataset-source", str(DATASET_PATH)])

print("🚀 Running benchmark...")
print(f"\nCommand: {' '.join(cmd)}")
print("\n" + "="*70 + "\n")

# Run the benchmark
result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print("\n" + "="*70)
    print("✅ Benchmark completed successfully!")
    print("="*70)
else:
    print("\n" + "="*70)
    print(f"❌ Benchmark failed with exit code: {result.returncode}")
    print("="*70)

## 3. View Results in Databricks MLflow

After running the benchmark:

1. **Open Databricks workspace**
2. **Navigate to**: Machine Learning → Experiments
3. **Find your user folder**: `/Users/your.email@domain.com/`
4. **Open experiment**: `sql-translation-benchmark`
5. **Compare runs**: Select multiple runs and click "Compare"

### Tips

- **Grouped bar charts**: Select 2+ runs to see side-by-side comparisons
- **Metrics to compare**: `avg_compliance_score`, `syntax_valid_pct`, `best_practices_pct`
- **LLM traces**: View in each run's "Traces" tab for debugging